<a href="https://colab.research.google.com/github/ak9250/X2Face/blob/master/x2facedemoface2face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install http://download.pytorch.org/whl/cu80/torch-0.2.0.post3-cp27-cp27mu-manylinux1_x86_64.whl

In [0]:
!wget url to your source image

In [0]:
!pip install autocrop

In [0]:
!autocrop -i /content/path to source image/ -o /content/cropped/ -w 256 -H 256 --facePercent 70

In [0]:
import torch
print(torch.__version__)

In [0]:
!pip install torchvision==0.2.0 --no-deps --no-cache-dir

In [0]:
!git clone https://github.com/oawiles/X2Face.git

In [0]:
cd X2Face/

In [0]:
cd UnwrapMosaic/

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import os
import torch
from PIL import Image
from torch.autograd import Variable
from UnwrappedFace import UnwrappedFaceWeightedAverage, UnwrappedFaceWeightedAveragePose
import torchvision
from torchvision.transforms import ToTensor, Compose, Scale

In [0]:
def run_batch(source_images, pose_images, requires_grad=False, volatile=False):
    return model(pose_images, *source_images)

In [0]:
from functools import partial
import pickle
import sys

pickle.load = partial(pickle.load, encoding="latin1")
pickle.Unpickler = partial(pickle.Unpickler, encoding="latin1")

def torch_load_model(path):
    if sys.version_info[0] < 3:
        if torch.cuda.is_available():
            state_dict = torch.load(path)
        else:
            state_dict = torch.load(path, map_location='cpu')
    else:
        if torch.cuda.is_available():
            state_dict = torch.load(path, pickle_module=pickle)
        else:
            state_dict = torch.load(path, map_location='cpu', pickle_module=pickle)
    return state_dict

In [0]:
!wget http://www.robots.ox.ac.uk/~vgg/research/unsup_learn_watch_faces/release_x2face_eccv.zip

In [0]:
!unzip release_x2face_eccv.zip

In [0]:
BASE_MODEL = 'release_models/' # Change to your path
state_dict = torch.load(BASE_MODEL + 'x2face_model.pth')

model = UnwrappedFaceWeightedAverage(output_num_channels=2, input_num_channels=3, inner_nc=128)
model.load_state_dict(state_dict['state_dict'])
model = model.cuda()

model = model.eval()

In [0]:
driver_path = '/content/X2Face/UnwrapMosaic/examples/Taylor_Swift/1.6/nuBaabkzzzI/'
source_path = '/content/cropped/'

driver_imgs = [driver_path + d for d in sorted(os.listdir(driver_path))][0:8] # 8 driving frames
source_imgs  = [source_path + d for d in sorted(os.listdir(source_path))][0:1] # 1 source frames

In [0]:
def load_img(file_path):
    img = Image.open(file_path)
    transform = Compose([Scale((256,256)), ToTensor()])
    return Variable(transform(img)).cuda()

In [0]:
# Driving the source image with the driving sequence
source_images = []
for img in source_imgs:
    source_images.append(load_img(img).unsqueeze(0).repeat(len(driver_imgs), 1, 1, 1))
    
driver_images = None
for img in driver_imgs:
    if driver_images is None:
        driver_images = load_img(img).unsqueeze(0)
    else:
        driver_images = torch.cat((driver_images, load_img(img).unsqueeze(0)), 0)

# Run the model for each
result = run_batch(source_images, driver_images)
result = result.clamp(min=0, max=1)
img = torchvision.utils.make_grid(result.cpu().data)
    
# Visualise the results
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 24.
fig_size[1] = 24.
plt.rcParams["figure.figsize"] = fig_size
plt.axis('off')
plt.savefig("test.png")

result_images = img.permute(1,2,0).numpy()
driving_images = torchvision.utils.make_grid(driver_images.cpu().data).permute(1,2,0).numpy()
print("The results is: ")
plt.imshow(np.vstack((result_images, driving_images)))